In [1]:
import pandas
import sklearn.ensemble
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn import datasets, linear_model
import numpy as np
import time
import datetime

1. Считайте таблицу с признаками из файла features.csv с помощью кода, приведенного выше. Удалите признаки, связанные с итогами матча (они помечены в описании данных как отсутствующие в тестовой выборке).

In [2]:
features = pandas.read_csv('./features.csv', index_col='match_id')
Y = features[['duration', 'radiant_win','tower_status_radiant','tower_status_dire','barracks_status_radiant','barracks_status_dire']]
X = features.drop(columns=['duration', 'radiant_win','tower_status_radiant','tower_status_dire','barracks_status_radiant','barracks_status_dire'])

X_test = pandas.read_csv('./features_test.csv', index_col='match_id')

2. Проверьте выборку на наличие пропусков с помощью функции count(), которая для каждого столбца показывает число заполненных значений. Много ли пропусков в данных? Запишите названия признаков, имеющих пропуски, и попробуйте для любых двух из них дать обоснование, почему их значения могут быть пропущены.

Всего матчей 97230. Ниже представлены признаки с пропусками и их количество.

In [3]:
for column in X:
    if X[column].count() < 97230:
        print(column + ":", end = " ")
        print(97230 - X[column].count())

first_blood_time: 19553
first_blood_team: 19553
first_blood_player1: 19553
first_blood_player2: 43987
radiant_bottle_time: 15691
radiant_courier_time: 692
radiant_flying_courier_time: 27479
radiant_first_ward_time: 1836
dire_bottle_time: 16143
dire_courier_time: 676
dire_flying_courier_time: 26098
dire_first_ward_time: 1826


3. Замените пропуски на нули с помощью функции fillna(). 

In [4]:
X = X.fillna(0)

4. Какой столбец содержит целевую переменную? Запишите его название.

In [5]:
y = Y["radiant_win"]

5. Забудем, что в выборке есть категориальные признаки, и попробуем обучить градиентный бустинг над деревьями на имеющейся матрице "объекты-признаки". Зафиксируйте генератор разбиений для кросс-валидации по 5 блокам (KFold), не забудьте перемешать при этом выборку (shuffle=True), поскольку данные в таблице отсортированы по времени, и без перемешивания можно столкнуться с нежелательными эффектами при оценивании качества. Оцените качество градиентного бустинга (GradientBoostingClassifier) с помощью данной кросс-валидации, попробуйте при этом разное количество деревьев (как минимум протестируйте следующие значения для количества деревьев: 10, 20, 30).

In [6]:
kf = sklearn.model_selection.KFold(n_splits=5, shuffle=True)

n_est = [10, 20, 30, 40, 50, 60]
for i in n_est:
    clf = sklearn.ensemble.GradientBoostingClassifier(n_estimators=i)
    start_time = datetime.datetime.now()
    score = cross_val_score(clf, X, y, cv=kf, scoring='roc_auc')
    print('Score:', np.mean(score), end=" ")
    print('Time elapsed:', datetime.datetime.now() - start_time)

Score: 0.6654545784998508 Time elapsed: 0:00:17.967523
Score: 0.6827753235965335 Time elapsed: 0:00:32.290937
Score: 0.6898725167014496 Time elapsed: 0:00:48.647249
Score: 0.6940015727520114 Time elapsed: 0:01:02.774887
Score: 0.6978576573970536 Time elapsed: 0:01:16.734847
Score: 0.7003294559662276 Time elapsed: 0:01:31.726160
